In [1]:
[x] no hypen in T-E
2. Goal should be in Goal
3. Use .. instead of "things to note..." for sphinx to create a "note"
    - Compress to a sentence or two
    - grammatical error in the note section
4. ToC 
    - kinda okay
    - 6. Tutorial part -> make it more telling
5. make change to rst
6. "Transformer" arch
7. link meta ai llama link in "llama-2"
8. "A lot is already..." is not a good statement, be more useful to the user
9. "we consider llama2" should not be used multiple places
10. Link llama2 to its proper place
11. More about the model is out of context
    - have a diagram about going from decoder to llama arch
    - link to a good llama tutorial
    - 
12. Huggingface is not the "go to" place (probably NeMo is)
13. Have all those imports/functions in `te_llama.py` file maybe
    - encapsulate the imports in `te_llama.py` or maybe have another utils.py
    - Average time over steps 
    - Could have a building table 
14. How to run with `FP8` as compared to `BF16` case 
    - Show how TELinear are run in `FP8` precision
    - From utils, do the funcs. The main difference is in showing how FP8 version is run
    - compress the code that actually trains/finetunes
15. Add the explanation for the the improvement 1
    - Be more formal
    - screenshots from nsys aren't great. The profiles could be hard to read. Annotate? 
    - Show the llama decoder vs te llama decoder in the profile, show cpu vs gpu activity (remove extra gpu info)
    - Could show that fp8 gemm is shorter than bf16 shorter in the annotation
        - gemms fp8 are faster but there are casts involved
        - Because of the CPU work
16. Nice segue into using larger layers in TE
17. (N in layernorm should be capital), name of `Attention` and `MLP` are wrong as per TE
18. Maximum thing that is provided is the full TransformerLayer
    - substituting the full TransformerLayer gives us good results
19. Introduce the transformerLayer after the Full transformerlayer intro
20. REmove the `MyGPT` thingy
21. Linking TE docs as maybe note
    - Here we're explorign the options needed for Llama, but TL has more options that are explained further in the API docs
22. "bunch of options" is not formal
23. Add the `te_llama.py` explained to `TransforerLayer` options explained
    - no way to use the `transformerlayer` directly in HF models but the options are kinda explainable.
    - the relevant options are explained as follows
    - The names of the weights are different, so need to take care of remapping
24. mapping of the weights is okay, add the links to the modules from hf llama modeling py
    - instead saying that we're just replacing the layers here.
25. monkey patching function showed in the tutorial as well (as excerpts)
26. "this .. runs .. Bf16" should be in a text somewhere. 
    - with that we have our next speedup
    - we see that even with BF16, it's still faster
27. Final version of the table is okay
28. change the ToC accordingly
29. more mindful of rst
    - newlines before and after the lists
    - rst is more syntax demanding
30. 


SyntaxError: unterminated string literal (detected at line 30) (2359419807.py, line 30)

# Accelerate HF Llama model with TransformerEngine

<div class="alert alert-info">

<b>Goal</b>

This tutorial showcases three incrementally efficient ways to use [TransformerEngine library](https://github.com/NVIDIA/TransformerEngine) to finetune (full) a Llama2 model from [Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b-hf).

</div>


<div class="alert alert-info">

<b>Note</b>
    
This tutorial showcases finetuning a full 7B Llama2 model (https://huggingface.co/meta-llama/Llama-2-7b-hf) on h100 GPUs (which have 80GB of HBM). Therefore, running the following individual portions (Baseline, Improvement 1, Improvement 2 and Improvement 3) for perf benchmarking will require restarting the Jupyter notebook kernel each time.

</div>


## Table of contents
1. From "Transformer" to "Llama"
2. Hugging Face's `LlamaModel`
    - Hugging Face's `LlamaDecoderLayer`
3. TransformerEngine's `TransformerLayer`
    - `TransformerLayer` options explained
4. Necessary Imports
5. [Baseline] Running HF `LlamaModel` (Precision: `BF16`)
6. [Improvement 1] Replace `nn.Linear` with TE's `Linear` layers (Precision: FP8)
7. [Improvement 2] Replace HF's `LlamaDecoderLayer` with TE's `TransformerLayer` (Precision: `BF16`)
8. [Improvement 3] Replace HF's `LlamaDecoderLayer` with TE's `TransformerLayer` (Precision: `FP8`)
9. Benchmarks revisited and Conclusion

### Understanding the aberration with Improvement 2 (CPU Overheads)

<figure align="center">
<img src="media/profile_hf_llama_bf16.png" width="100%">
    <figcaption> Fig 1: Profile of HF Llama baseline implementation. (Top) A cross-section of the CPU/GPU activity showing a single transformer layer (mainly "SelfAttention" and "MLP" modules) forward. (Middle) An emphasized cross-section of a portion of the "SelfAttention" module that shows the "Q", "K" and "V" projection operations (basically `nn.linear` layers). (Bottom) A further emphasized cross-section of only the "Q" projection operation which shows the corresponding CPU and GPU activity for a single `nn.Linear` layer. </figcaption>
</figure>


<figure align="center">
<img src="media/profile_hf_llama_fp8.png" width="100%">
    <figcaption> Fig 1: Profile of HF Llama with "Improvement 1" (replacing `nn.Linear` layers with TE's `Linear` layers) implementation. (Top) A cross-section of the CPU/GPU activity showing a single transformer layer (mainly "MultiheadAttention" and "LayerNormMLP" modules) forward. (Middle) An emphasized cross-section of a portion of the "MultiheadAttention" module that shows the Q, K and V projection operations (basically TE's `Linear` layers). (Bottom) A further emphasized cross-section of only the Q projection operation which shows the CPU and GPU activity for a single TE's `Linear` layer.</figcaption>
</figure>

<div class="alert alert-light">

<b>Insight</b>
    
In the profiles above, whenever the GPU activity is absent, it generally indicates that the GPU is waiting for CPU to dispatch a kernel i.e. the GPU is idle and waiting for more work. In general, we'd want the GPU to be active as much as possible and wait less for the CPU.
    
One thing clearly noticeable is that the GPU kernels in the "baseline" implementation occupy GPU more of the time than the "Improvement 1" (replacing `nn.Linear` with TE's `Linear`) implementation. Let's dig into why that is the case.

</div>

To simplify the information from the above profiles, consider the following table that compares the two implementations:

|                                                     |              |                        |              |                   | Baseline (microseconds) | Improvement 1 (replace `nn.Linear` with TE's `Linear`)(microseconds) | Speedup |
|-----------------------------------------------------|--------------|------------------------|--------------|-------------------|-------------------------|----------------------------------------------------------------------|---------|
| Single transformer layer forward ("attn" and "mlp") |              |                        |              |                   | 2443                    | 6299                                                                 | -       |
|                                                     | "attn" layer |                        |              |                   | 1470                    | 3842                                                                 | -       |
|                                                     |              | Q, K and V projections |              |                   | 326                     | 2027                                                                 | -       |
|                                                     |              |                        | Q projection |                   | 117                     | 1001                                                                 | -       |
|                                                     |              |                        |              | Amax/Scale update | -                       | 72                                                                   | -       |
|                                                     |              |                        |              | Buffer allocation | -                       | 49                                                                   | -       |
|                                                     |              |                        |              | Cast+Transpose    | -                       | 35 (10 + 25)                                                         | -       |
|                                                     |              |                        |              | MatMul            | 106                     | 52                                                                   | 2.03x   |
|                                                     | "mlp" layer  |                        |              |                   | 790                     | 2057                                                                 | -       |


Now let's make a few observations:

1. For a single transformer layer, Improvement 1 implementation (`nn.Linear` layers replaced with TE's `Linear` layers and with FP8 precision) takes more time than the baseline implementation (BF16 precision). 
2. If we keep zooming in the profile to individual "attn" (`SelfAttention` for baseline and `MultiheadAttention` for Improvement 1) or "mlp" (`MLP` for baseline or `LayerNormMLP` for Improvement 1) layers, the trend is similar that Improvement 1 is slower than the baseline implementation.
3. At its core the `Linear` layers in Improvement 1 are slower than the `nn.Linear` layers in the baseline implementation (1001 microseconds vs 117 microseconds, i.e. _slower by a factor of **8.5x**_)

#### Why is TE's `Linear` slower than `nn.Linear` layer?

If we look closely, TE's `Linear` layer contains more kernels for the following tasks: 
1. Amax and scale update 
2. FP8 weights and transpose buffer allotment
3. Cast+Transpose kernels for inputs and weights (to cast BF16 inputs and weights to their FP8 counterparts)
4. Matrix Multiplication kernel (in FP8 precision)

All those kernels are pretty short for the workload for the current finetuning tutorial (`batch_size=8`, `max_seq_length=256`).

Compare this to `nn.Linear` layer that contains only a Matrix Multiplication kernel (in BF16 precision

<div class="alert alert-success">

<b>Insight</b>
    
In the Improvement 1, the Matrix Multiplication operation in FP8 precision itself is faster than the Matrix Multiplication in BF16 precision in the baseline implementation!
    
What if we could force the GPU to spend more time in Matrix Multiplication?

</div>

#### How to make TE's `Linear` faster than `nn.Linear` layer?

As we noted earlier, the workload for the current finetuning tutorial (`batch_size=8`, `max_seq_length=256`) is a bit short and therefore isn't able to fully utilize the capability of TE's `Linear` layers.

Generally, we'd want to increase the workload so that GPU is active more of the time than the CPU.

As a small experiment, let's see how the profiles look like for the "Q" projection operation when we increase the `batch_size` from `8` to `128`. Since this result in GPU running of memory, let's simultaneously decrease the size of our model from 32 layers to just `4` (i.e. `config=num_hidden_layers=4`).


<figure align="center">
<img src="media/profile_hf_llama_bf16_bs_128.png" width="100%">
    <figcaption> Fig 1: (baseline implementation) Profile of the "Q" projection, especially the `nn.Linear` layer when batch_size=128 and num_hidden_layers=4. </figcaption>
</figure>


<figure align="center">
<img src="media/profile_hf_llama_fp8_bs_128.png" width="100%">
    <figcaption> Fig 1: (Improvement 1 implementation) Profile of the "Q" projection, especially the TE's `Linear` layer when  batch_size=128 and num_hidden_layers=4. </figcaption>
</figure>



## From "Transformer" to "Llama" 

<figure align="center">
<img src="media/transformer_llama.png" width="50%">
    <figcaption> Fig 1: Llama visualized as a transformer. (generated with <a href="https://catalog.ngc.nvidia.com/orgs/nvidia/teams/ai-foundation/models/sdxl">Nvidia's AI-foundation models</a>)</figcaption>
</figure>

A flashback:
- 2017: ["Attention Is All You Need"](https://arxiv.org/abs/1706.03762) paper introduced pioneering "Transformer" architecture and changed the NLP field forever.
- 2018-2020: Emergence of GPT model series that showed causal decoder architectures are great fit for pretraining, few-shot and zero-shot learning.
- Fast forward to 2023-2024: Following GPT-3/GPT-4 success stories, researchers and companies raced to produce the next best pretrained model that could further be finetuned for application-specific use-cases. 
- One of the latest in this line of pretrained models which is also open source is Meta's [Llama 2](https://llama.meta.com/llama2) models (Large Language Model Meta AI). 
    - These models range from 7B to 65B parameters.
    - LLaMA 2 was pretrained on 2 trillion tokens.

For more information on Llama 2 consider reading the [Huggingface tutorial](https://huggingface.co/blog/llama2). As a quick summary, here are some of the important differences b/w the conventional decoder architecture vs Llama2:

[image about decoder arch vs llama2]

1. Decoder only model (causal language modeling and next word prediction)
2. RMSNorm in place of the LayerNorm
3. SwiGLU activation function
4. RoPE as positional embeddings 
5. Grouped Query Attention
6. Trained on 4K context length

## Hugging Face's `LlamaModel`
Hugging Face provides an open-source implementation of `Llama` model in [`modeling_llama.py`](https://github.com/huggingface/transformers/blob/3d2900e829ab16757632f9dde891f1947cfc4be0/src/transformers/models/llama/modeling_llama.py#L4).

Here's a block diagram that shows how Llama model is implemented in the Hugging Face repo. Notice the modular encapsulated form and `LlamaDecoderLayer` at the core of the model implementation. This core layer chunk is targeted for optimizations in a couple of the improvements later in this tutorial (Improvement 2 and Improvement 3). 

<figure align="center">
<img src="media/llama_for_causal_lm.png" width="40%">
    <figcaption> Fig 2: Causal Llama Model Block Diagram. </figcaption>
</figure>

The above diagram translates to the following text output of the model in pytorch. Notice that the core of the model has 32 `LlamaDecoderLayer`s. 

```
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)
```

#### HF `LlamaDecoderLayer`

Let's take a closer look at `LlamaDecoderLayer`. It's composed of `input_layernorm`, `self_attn`, `post_attention_layernorm` and `mlp` modules. Each module has associated weights as shown in the diagram.

<figure align="center">
<img src="media/llama_zoom.png" width="70%">
    <figcaption> Fig 3: Causal Llama Model Block Diagram (<a href="https://github.com/huggingface/transformers/blob/e770f0316d2a9b787c9d1440f204fcb65e176682/src/transformers/models/llama/modeling_llama.py#L695">LlamaDecoderLayer</a>). </figcaption>
</figure>

- [ ] Explain the layers a bit (e.g. Swiglu (up_proj and gate_proj))

## [Baseline] Running HF `LlamaModel` (Precision: `BF16`)

Llama2 weights are loaded into the Hugging Face native implementation `LlamaForCausalLM` (refer to [modeling_llama.py](https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py)). 

`batch_size` is `8` and precision is `BF16`

The `LlamaDecoderLayer` is left unchanged in the baseline as follows:

<figure align="center">
<img src="media/llamadecoderlayer.png" width="30%">
    <figcaption> Fig 4: Revisiting "LlamaDecoderLayer". </figcaption>
</figure>


In [7]:
%reset -f

In [9]:
import torch
torch.cuda.memory_allocated(device="cuda")

54270198784

In [3]:
# Minimize the bloat by wrapping all the imports in a function and return
from utils import *


## Default hyperparams, also defined in `utils.py` in class `Hyperparameters`
# hyperparams.dataset_name = "timdettmers/openassistant-guanaco"
# hyperparams.dataset_text_field = "text"
# hyperparams.learning_rate = 1.41e-5
# hyperparams.batch_size = 8
# hyperparams.max_seq_length = 256
# hyperparams.gradient_accumulation_steps = 1
# hyperparams.num_training_steps=10


## !!! `model_name` attr must point to the location of the model weights !!!
hyperparams.model_name = "/bei/datasets/llama-7bf-hf/" # <== Add model weight location here
hyperparams.mixed_precision = "bf16"


## Init the model and accelerator wrapper
model = init_baseline_model(hyperparams)
accelerator, model, optimizer, train_dataloader, lr_scheduler = wrap_with_accelerator(model, hyperparams)


## Finetune the model
finetune_model(model, hyperparams, accelerator, train_dataloader, optimizer, lr_scheduler)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecat

State: Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: bf16



/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9846/9846 [00:00<00:00, 11108.04 examples/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 63.21 GiB of which 52.75 MiB is free. Process 30030 has 63.15 GiB memory in use. Of the allocated memory 60.88 GiB is allocated by PyTorch, and 1.55 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [7]:
# Fine-tune the model
model.train()
completed_steps = 0
total_loss = 0
optimizer.zero_grad()

for _ in range(10):
    if completed_steps >= num_training_steps:
        break
    for step, batch in enumerate(train_dataloader):
        start_time = time.time()
        with accelerator.accumulate(model):
            outputs = model(**batch)
            loss = outputs.loss
            print(f"Step {step}: loss: {loss.item()}, batch shape: {batch['input_ids'].shape}, peak gpu mem: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")
            total_loss += loss.detach().float()
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        if accelerator.sync_gradients:
            completed_steps += 1

        end_time = time.time()
        total_time = end_time - start_time
        print(f"Step {step} time {total_time}")
        accelerator.log({"batch_time": total_time, "input_ids": batch["input_ids"].cpu().numpy(), "attention_mask": batch["attention_mask"].cpu().numpy()})
        start_time = end_time

        if completed_steps >= num_training_steps:
            break

accelerator.end_training()

Step 0: loss: 1.742400050163269, batch shape: torch.Size([8, 256]), peak gpu mem: 25.39 GB
Step 0 time 0.7563114166259766
Step 1: loss: 2.0216784477233887, batch shape: torch.Size([8, 256]), peak gpu mem: 63.09 GB
Step 1 time 0.2901787757873535
Step 2: loss: 2.2615396976470947, batch shape: torch.Size([8, 256]), peak gpu mem: 63.09 GB
Step 2 time 0.2868986129760742
Step 3: loss: 2.092174530029297, batch shape: torch.Size([8, 256]), peak gpu mem: 63.09 GB
Step 3 time 0.289764404296875
Step 4: loss: 2.0941758155822754, batch shape: torch.Size([8, 256]), peak gpu mem: 63.09 GB
Step 4 time 0.28885555267333984
Step 5: loss: 2.010141372680664, batch shape: torch.Size([8, 256]), peak gpu mem: 63.09 GB
Step 5 time 0.287872314453125
Step 6: loss: 2.3474209308624268, batch shape: torch.Size([8, 256]), peak gpu mem: 63.09 GB
Step 6 time 0.2875847816467285
Step 7: loss: 1.8907099962234497, batch shape: torch.Size([8, 256]), peak gpu mem: 63.09 GB
Step 7 time 0.28859663009643555
Step 8: loss: 1.920

| Models                                                      | Precision | Step Time (or ms per batch) | Speedup (over baseline) |
|-------------------------------------------------------------|-----------|-----------------------------|-------------------------|
| HF (baseline)                                               | BF16      | 288                         | 1                     

## [Improvement 1] Replace `nn.Linear` with TE's `Linear` layers (Precision: `FP8`)

[Hugging Face accelerate](https://github.com/huggingface/accelerate) provides a [`convert_model`](https://github.com/huggingface/accelerate/blob/97d2168e5953fe7373a06c69c02c5a00a84d5344/src/accelerate/utils/transformer_engine.py#L24) method to replace `torch.nn.Linear` layers with `transformer_engine.pytorch.module.Linear` layers. The following diagram illustrates this visually.

<figure align="center">
<img src="media/llamadecoderlayer_replace_with_telinear.png" width="70%">
    <figcaption> Fig 5: Replacing "nn.Linear" with "TE.Linear". </figcaption>
</figure>


This is the most straightforward way to use TransformerEngine's `FP8` precision during training/finetuning for HF Llama model. Notice that the entire `LlamaDecoderLayer` is mostly left unchanged, it's only the `nn.Linear` layers that get replaced with `TE.Linear` layers. After replacing, these layers can be run in `FP8` precision.



In [4]:
# Minimize the bloat by wrapping all the imports in a function and return
from utils import *


## Default hyperparams, also defined in `utils.py` in class `Hyperparameters`
# hyperparams.dataset_name = "timdettmers/openassistant-guanaco"
# hyperparams.dataset_text_field = "text"
# hyperparams.learning_rate = 1.41e-5
# hyperparams.batch_size = 8
# hyperparams.max_seq_length = 256
# hyperparams.gradient_accumulation_steps = 1
# hyperparams.num_training_steps=10


## !!! `model_name` attr must point to the location of the model weights !!!
hyperparams.model_name = "/ckpt/llama-7bf-hf/" # <== Add model weight location here
hyperparams.mixed_precision = "fp8"


## Init the model and accelerator wrapper
model = init_baseline_model(hyperparams)
accelerator, model, optimizer, train_dataloader, lr_scheduler = wrap_with_accelerator(model, hyperparams)


## Finetune the model
finetune_model(model, hyperparams, accelerator, train_dataloader, optimizer, lr_scheduler)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.24s/it]


In [7]:
# Train the model
model.train()
completed_steps = 0
total_loss = 0
optimizer.zero_grad()

for _ in range(10):
    if completed_steps >= num_training_steps:
        break
    for step, batch in enumerate(train_dataloader):
        start_time = time.time()
        with accelerator.accumulate(model):
            outputs = model(**batch)
            loss = outputs.loss
            print(f"Step {step}: loss: {loss.item()}, batch shape: {batch['input_ids'].shape}, peak gpu mem: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")
            total_loss += loss.detach().float()
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        if accelerator.sync_gradients:
            completed_steps += 1

        end_time = time.time()
        total_time = end_time - start_time
        print(f"Step {step} time {total_time}")
        accelerator.log({"batch_time": total_time, "input_ids": batch["input_ids"].cpu().numpy(), "attention_mask": batch["attention_mask"].cpu().numpy()})
        start_time = end_time

        if completed_steps >= num_training_steps:
            break

accelerator.end_training()

Step 0: loss: 1.7408778667449951, batch shape: torch.Size([8, 256]), peak gpu mem: 30.95 GB
Step 0 time 1.494196891784668
Step 1: loss: 2.0046780109405518, batch shape: torch.Size([8, 256]), peak gpu mem: 68.52 GB
Step 1 time 0.926837682723999
Step 2: loss: 2.2610206604003906, batch shape: torch.Size([8, 256]), peak gpu mem: 68.52 GB
Step 2 time 0.30959415435791016
Step 3: loss: 2.0951247215270996, batch shape: torch.Size([8, 256]), peak gpu mem: 68.55 GB
Step 3 time 0.3102715015411377
Step 4: loss: 2.0994796752929688, batch shape: torch.Size([8, 256]), peak gpu mem: 68.55 GB
Step 4 time 0.30820536613464355
Step 5: loss: 2.005664587020874, batch shape: torch.Size([8, 256]), peak gpu mem: 68.55 GB
Step 5 time 0.3083922863006592
Step 6: loss: 2.3457205295562744, batch shape: torch.Size([8, 256]), peak gpu mem: 68.55 GB
Step 6 time 0.30889034271240234
Step 7: loss: 1.8973335027694702, batch shape: torch.Size([8, 256]), peak gpu mem: 68.55 GB
Step 7 time 0.30847978591918945
Step 8: loss: 1

Based on the above run, the performance of the baseline HF Llama 2 implementation is as follows:

| Models                                                      | Precision | Step Time (or ms per batch) | Speedup (over baseline) |
|-------------------------------------------------------------|-----------|-----------------------------|-------------------------|
| HF (baseline)                                               | BF16      | 288                         | 1                       |
| HF (replace `nn.Linear` with `TE.Linear`)                   | FP8       | 307                         | _**0.94**_                 

The performance with TE `Linear` layers has actually decreased. Let's try to understand the reason of the slow down. 

### Understanding the aberration with Improvement 2 (CPU Overheads)


<figure align="center">
<img src="media/profile_hf_llama_bf16.png" width="100%">
    <figcaption> Fig 1: Profile of HF Llama baseline implementation. (Top) A cross-section of the CPU/GPU activity showing a single transformer layer (mainly "SelfAttention" and "MLP" modules) forward. (Middle) An emphasized cross-section of a portion of the "SelfAttention" module that shows the "Q", "K" and "V" projection operations (basically `nn.linear` layers). (Bottom) A further emphasized cross-section of only the "Q" projection operation which shows the corresponding CPU and GPU activity for a single `nn.Linear` layer. </figcaption>
</figure>


<figure align="center">
<img src="media/profile_hf_llama_fp8.png" width="100%">
    <figcaption> Fig 1: Profile of HF Llama with "Improvement 1" (replacing `nn.Linear` layers with TE's `Linear` layers) implementation. (Top) A cross-section of the CPU/GPU activity showing a single transformer layer (mainly "MultiheadAttention" and "LayerNormMLP" modules) forward. (Middle) An emphasized cross-section of a portion of the "MultiheadAttention" module that shows the Q, K and V projection operations (basically TE's `Linear` layers). (Bottom) A further emphasized cross-section of only the Q projection operation which shows the CPU and GPU activity for a single TE's `Linear` layer.</figcaption>
</figure>

<div class="alert alert-light">

<b>Insight</b>
    
In the profiles above, whenever the GPU activity is absent, it generally indicates that the GPU is waiting for CPU to dispatch a kernel i.e. the GPU is idle and waiting for more work. In general, we'd want the GPU to be active as much as possible and wait less for the CPU.
    
One thing clearly noticeable is that the GPU kernels in the "baseline" implementation occupy GPU more of the time than the "Improvement 1" (replacing `nn.Linear` with TE's `Linear`) implementation. Let's dig into why that is the case.

</div>

To simplify the information from the above profiles, consider the following table that compares the two implementations:

|                                                     |              |                        |              |                   | Baseline (microseconds) | Improvement 1 (replace `nn.Linear` with TE's `Linear`)(microseconds) | Speedup |
|-----------------------------------------------------|--------------|------------------------|--------------|-------------------|-------------------------|----------------------------------------------------------------------|---------|
| Single transformer layer forward ("attn" and "mlp") |              |                        |              |                   | 2443                    | 6299                                                                 | -       |
|                                                     | "attn" layer |                        |              |                   | 1470                    | 3842                                                                 | -       |
|                                                     |              | Q, K and V projections |              |                   | 326                     | 2027                                                                 | -       |
|                                                     |              |                        | Q projection |                   | 117                     | 1001                                                                 | -       |
|                                                     |              |                        |              | Amax/Scale update | -                       | 72                                                                   | -       |
|                                                     |              |                        |              | Buffer allocation | -                       | 49                                                                   | -       |
|                                                     |              |                        |              | Cast+Transpose    | -                       | 35 (10 + 25)                                                         | -       |
|                                                     |              |                        |              | MatMul            | 106                     | 52                                                                   | 2.03x   |
|                                                     | "mlp" layer  |                        |              |                   | 790                     | 2057                                                                 | -       |


Now let's make a few observations:

1. For a single transformer layer, Improvement 1 implementation (`nn.Linear` layers replaced with TE's `Linear` layers and with FP8 precision) takes more time than the baseline implementation (BF16 precision). 
2. If we keep zooming in the profile to individual "attn" (`SelfAttention` for baseline and `MultiheadAttention` for Improvement 1) or "mlp" (`MLP` for baseline or `LayerNormMLP` for Improvement 1) layers, the trend is similar that Improvement 1 is slower than the baseline implementation.
3. At its core the `Linear` layers in Improvement 1 are slower than the `nn.Linear` layers in the baseline implementation (1001 microseconds vs 117 microseconds, i.e. _slower by a factor of **8.5x**_)

#### Why is TE's `Linear` slower than `nn.Linear` layer?

If we look closely, TE's `Linear` layer contains more kernels for the following tasks: 
1. Amax and scale update 
2. FP8 weights and transpose buffer allotment
3. Cast+Transpose kernels for inputs and weights (to cast BF16 inputs and weights to their FP8 counterparts)
4. Matrix Multiplication kernel (in FP8 precision)

While the GPU is idle, it's usually waiting for CPU to finish doing its work and dispatch a kernel that can run on the GPU! Further, all those kernels are pretty short for the workload for the current finetuning tutorial (`batch_size=8`, `max_seq_length=256`). Therefore, overall the time taken by the TE's `Linear` layer is **1001 microseconds**. 

Compare this to `nn.Linear` layer (in Fig: ???) which contains only a Matrix Multiplication kernel (in BF16 precision). Almost all of the work inside the linear layer is running that kernel. Further, as the GPU doesn't have to wait for the CPU for more kernels, the `nn.Linear` layer itself takes less time to run, only **117 microseconds**, almost a tenth of the fraction of TE's `Linear` layer. 

<div class="alert alert-success">

<b>Insight</b>
    
Note that in Improvement 1, the Matrix Multiplication operation in FP8 precision itself is faster than the Matrix Multiplication in BF16 precision in the baseline implementation!
    
What if we could force the GPU to spend more time in Matrix Multiplication?

</div>

#### How to make TE's `Linear` faster than `nn.Linear` layer?

As we noted earlier, the workload for the current finetuning tutorial (`batch_size=8`, `max_seq_length=256`) is a bit short and therefore isn't able to fully utilize the capability of TE's `Linear` layers.

Generally, we'd want to increase the workload so that GPU is active more of the time than the CPU.

As a small experiment, let's see how the profiles look like for the "Q" projection operation when we increase the `batch_size` from `8` to `128`. Since this result in GPU running of memory, let's simultaneously decrease the size of our model from 32 layers to just `4` (i.e. `config=num_hidden_layers=4`).


<figure align="center">
<img src="media/profile_hf_llama_bf16_bs_128.png" width="100%">
    <figcaption> Fig 1: (baseline implementation) Profile of the "Q" projection, especially the `nn.Linear` layer when batch_size=128 and num_hidden_layers=4. </figcaption>
</figure>


<figure align="center">
<img src="media/profile_hf_llama_fp8_bs_128.png" width="100%">
    <figcaption> Fig 1: (Improvement 1 implementation) Profile of the "Q" projection, especially the TE's `Linear` layer when  batch_size=128 and num_hidden_layers=4. </figcaption>
</figure>



### Understanding the aberration with Improvement 2 (CPU Overheads)

TransformerEngine's `Linear` layer has to do some extra work (in python, so on CPU) before it can issue `FP8` version of GEMMs (matrix-multiplies from the linear layers) to the GPU.




2. So, if batch-size and/or sequence-length are small enough, the GEMM will be smaller. The GEMMs on GPU therefore usually finish early than the CPU can issue them in succession. In this case, we say the the workload is CPU bound. Ideally we'd want the workload to be GPU bound, i.e. it should spend more time in the GPU to fully utilize its capability.
3. We can verify this by looking the profiles of the two workloads (using Nvidia Nsight Systems visualizer).
    - For the baseline case (with `nn.Linear` layers):
        ![baseline profile](media/baseline.png "baseline")
        
    - Improvement 1 - when `nn.Linear`s are replaced with `TE.Linear`s:
        ![replace linears](media/replace_nnlinear_with_telinear.png "replace linears")
        
    - As we can see, the GPU is busy comparatively less in the second case, compared to the first. CPU overhead is not letting 

But How do we alleviate this issue? 

To fully utilize the TransformerEngine's capabilities, the workload should be larger. In this present tutorial, the batch-size is 8 which is fine but sequence-length is 256 which is pretty low. If the sequence-length is large enough, even this case could be faster than the baseline.

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>") #Restart the kernel

In [1]:
from utils import *

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
hyperparams.model_name = "/bei/datasets/llama-7bf-hf"
config = AutoConfig.from_pretrained(hyperparams.model_name)

In [4]:
config.num_hidden_layers = 8

In [5]:
# make sure to use flash_attention to do iso comparison with TELlamaModel
config._attn_implementation = "flash_attention_2"
model = AutoModelForCausalLM.from_pretrained(
    hyperparams.model_name,
    config=config,
    torch_dtype=torch.bfloat16,
)
# Needed for the cases when using TELlamaForCausalLM. So adding here for 1:1 comparison
model.config.use_cache=False

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage

In [6]:
hyperparams.max_seq_length = 400
hyperparams.mixed_precision = "fp8"

In [7]:
accelerator, model, optimizer, train_dataloader, lr_scheduler = wrap_with_accelerator(model, hyperparams)

/perfhome/repos/2023/megatron/vicuna/hf/accelerate/src/accelerate/accelerator.py:387: UserWarning: `log_with=wandb` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")


State: Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp8



/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9846/9846 [00:00<00:00, 10146.56 examples/s]


In [8]:
finetune_model(model, hyperparams, accelerator, train_dataloader, optimizer, lr_scheduler)

Step 0: loss: 9.768940925598145 time: 0.5495994091033936 batch_size torch.Size([8, 400])
Step 1: loss: 10.110845565795898 time: 0.10734415054321289 batch_size torch.Size([8, 400])
Step 2: loss: 10.416016578674316 time: 0.10294985771179199 batch_size torch.Size([8, 400])
Step 3: loss: 10.218790054321289 time: 0.10591721534729004 batch_size torch.Size([8, 400])
Step 4: loss: 10.144618034362793 time: 0.10585546493530273 batch_size torch.Size([8, 400])
Step 5: loss: 10.026362419128418 time: 0.10659241676330566 batch_size torch.Size([8, 400])
Step 6: loss: 10.259310722351074 time: 0.10713028907775879 batch_size torch.Size([8, 400])
Step 7: loss: 9.968159675598145 time: 0.10583972930908203 batch_size torch.Size([8, 400])
Step 8: loss: 9.914514541625977 time: 0.10669207572937012 batch_size torch.Size([8, 400])
Step 9: loss: 9.677173614501953 time: 0.10484886169433594 batch_size torch.Size([8, 400])


In [8]:
finetune_model(model, hyperparams, accelerator, train_dataloader, optimizer, lr_scheduler)

Step 0: loss: 9.783175468444824 time: 1.321913242340088 batch_size torch.Size([8, 400])
Step 1: loss: 10.131692886352539 time: 0.7767603397369385 batch_size torch.Size([8, 400])
Step 2: loss: 10.436424255371094 time: 0.09566235542297363 batch_size torch.Size([8, 400])
Step 3: loss: 10.226944923400879 time: 0.09562826156616211 batch_size torch.Size([8, 400])
Step 4: loss: 10.1524658203125 time: 0.09576177597045898 batch_size torch.Size([8, 400])
Step 5: loss: 10.031890869140625 time: 0.09409642219543457 batch_size torch.Size([8, 400])
Step 6: loss: 10.274042129516602 time: 0.09509897232055664 batch_size torch.Size([8, 400])
Step 7: loss: 9.964439392089844 time: 0.10423588752746582 batch_size torch.Size([8, 400])
Step 8: loss: 9.933443069458008 time: 0.10542511940002441 batch_size torch.Size([8, 400])
Step 9: loss: 9.695547103881836 time: 0.10503625869750977 batch_size torch.Size([8, 400])


In [11]:
train_dataloader = enumerate(train_dataloader)

In [12]:
step, batch = next(train_dataloader)

In [13]:
batch['input_ids'].shape

torch.Size([8, 400])

In [14]:
batch['attention_mask'].shape

torch.Size([8, 400])

In [ ]:
batch['attention_mask'].shape

In [14]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [17]:
model.train()
total_loss = 0
optimizer.zero_grad()
# train_dataloader = enumerate(train_dataloader)

for _ in range(hyperparams.num_training_steps):
    step, batch = next(train_dataloader)
    start_time = time.time()
    with accelerator.accumulate(model):
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    end_time = time.time()
    total_time = end_time - start_time
    print(f"Step {step}: loss: {loss.item()} time: {total_time} batch_size {batch['input_ids'].shape}")

Step 3: loss: 10.114119529724121 time: 0.14879465103149414 batch_size torch.Size([8, 624])
Step 4: loss: 10.01146125793457 time: 0.20577788352966309 batch_size torch.Size([8, 1104])
Step 5: loss: 9.758525848388672 time: 0.2152695655822754 batch_size torch.Size([8, 1200])
Step 6: loss: 10.310785293579102 time: 0.19851088523864746 batch_size torch.Size([8, 1072])
Step 7: loss: 9.95548152923584 time: 0.10099005699157715 batch_size torch.Size([8, 448])
Step 8: loss: 9.752256393432617 time: 0.16744017601013184 batch_size torch.Size([8, 864])
Step 9: loss: 9.563212394714355 time: 0.14487099647521973 batch_size torch.Size([8, 704])
Step 10: loss: 10.125089645385742 time: 0.13337016105651855 batch_size torch.Size([8, 624])
Step 11: loss: 9.681853294372559 time: 0.1406705379486084 batch_size torch.Size([8, 672])
Step 12: loss: 10.049918174743652 time: 0.17294764518737793 batch_size torch.Size([8, 928])


## [Improvement 2] Replace HF's `LlamaDecoderLayer` with TE's `TransformerLayer` (Precision: `BF16`)

In addition to basic layers like `Linear` and `LayerNorm`, TransformerEngine offers larger modules like `MultiheadAttention` (combines "LayerNorm" and "Self Attention") and `LayerNormMLP` (combines "LayerNorm" and "MLP") that could replace their counterparts in the `LlamaDecoderLayer` and potentially provide more speedup. Further, TransformerEngine also offers a full `TransformerLayer` (which further combines `MultiheadAttention` and `LayerNormMLP` layers) which could be substituted for `LlamaDecoderLayer` (with careful mapping of the weights since the name of the weights are different for those two layers). Let's take a closer look at TransformerEngine's `TransformerLayer`. 

### TransformerEngine's `TransformerLayer`

At a higher level, TE's `TransformerLayer` could be visualized as an apt replacement for the `LlamaDecoderLayer`. But the internals of the `TransformerLayer` are organized a bit differently. 

<figure align="center">
<img src="media/tellamadecoderlayer.png" width="30%">
    <figcaption> Fig 6: TransformerEngine's `TransformerLayer` </figcaption>
</figure>

Just like Hugging Face's `LlamaDecoderLayer`, TransformerEngine's `TransformerLayer` encapsulates `self_attention` (as `MultiheadAttention`) and `mlp` (as `LayerNormMLP`). A major difference is that the two `Norm`s are included in the `MultiheadAttention` and `LayerNormMLP` layers as shown in the following output prompt:

```
TransformerLayer(
    (self_attention): MultiheadAttention(
      (layernorm_qkv): LayerNormLinear()
      (core_attention): DotProductAttention()
      (proj): Linear()
    )
    (layernorm_mlp): LayerNormMLP()
)
```

### `TransformerLayer` options explained

<div class="alert alert-info">

<b>Note</b>
    
Here, we go over some of the options in `TransformerLayer` that are needed for the tutorial. For a complete list of options, refer the [TransformerLayer API documentation](https://docs.nvidia.com/deeplearning/transformer-engine/user-guide/api/pytorch.html?highlight=transformerlayer#transformer_engine.pytorch.TransformerLayer).

</div>

In the accompanying `te_llama.py` file, `TELlamaDecoderLayer` is defined as a wrapper over TE's `TransformerLayer` with a few needed options that make `TransformerLayer` as a plug-in replacement for the HF's `LlamaDecoderLayer`.

```
class TELlamaDecoderLayer(te.pytorch.TransformerLayer):
    def __init__(self, config):
        super().__init__(
            config.hidden_size,
            config.intermediate_size,
            config.num_attention_heads,
            bias=False,
            layernorm_epsilon=config.rms_norm_eps,
            hidden_dropout=0,
            attention_dropout=0,
            fuse_qkv_params=False,
            normalization="RMSNorm",
            activation="swiglu",
            attn_input_format="bshd",
        )
        te_rope = RotaryPositionEmbedding(config.hidden_size//config.num_attention_heads)
        self.te_rope_emb = te_rope(max_seq_len=config.max_position_embeddings).cuda()
```

Here's a list summarizing each option briefly:

1. `hidden_size`: size of each input sample.
2. `ffn_hidden_size`: intermediate size to which samples are projected.
3. `num_attention_heads`: number of attention heads in the transformer layer.
4. `bias`: switch to add additive biases to the submodule layers.
5. `layernorm_epsilon`: a value added to the denominator of layer normalization for numerical stability. Default is `1e-5`.
6. `hidden_dropout`: dropout probability for the dropout op after FC2 layer (fully connected layer no. 2). Default is `0.1`.
7. `attention_dropout`: dropout probability for the dropout op during multi-head attention. Default is `0.1`. 
8. `fuse_qkv_params`:  if set to True, TransformerLayer module exposes a single fused parameter for query-key-value. This enables optimizations such as QKV fusion without concatentations/splits and also enables the argument fuse_wgrad_accumulation.
9. `normalization`: type of normalization applied. Default is `LayerNorm`.
10. `activation`: type of activation used in the MLP block. Default is `gelu`.
11. `attn_input_format`: controls whether the dimensions of the intermediate hidden states is 'batch first' ('bshd') or 'sequence first' ('sbhd'). `s` stands for the sequence length, `b` batch size, `h` the number of heads, `d` head size. Note that these formats are very closely related to the `qkv_format` in the `MultiHeadAttention` and `DotProductAttention` modules. 


Further, note that `RotaryPositionEmbedding` is defined as part of the TE's `TransformerLayer` itself since it expects this rope cache if RoPE is used in the model. 

### Comparing Hugging Face's `LlamaDecoderLayer` with TranformerEngine's `TransformerLayer`

Let's revisit how `LlamaDecoderLayer`s form the core of the decoder layer stack in HF's llama implementation:
```
ModuleList(
  (0-31): 32 x LlamaDecoderLayer(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
)
```

A major portion of the Hugging Face model implementation (`LlamaDecoderLayer`) could be potentially replaced with TransformerEngine's implementation.


### Mapping weights from HF's `LlamaDecoderLayer` to TE's `TransformerLayer`

Refer the accompanying file `te_llama.py` which provides a reference to create a Llama 2 model with TE's `TransformerLayer` after replacing HF's `LlamaDecoderLayer`.

Briefly, 
1. `TELlamaDecoderLayer` is added as a wrapper for `TransformerLayer`. 
```
class TELlamaDecoderLayer(te.pytorch.TransformerLayer):
    """
    Wrapper class over TE's `TransformerLayer`. This makes the wrapper very
    similar to HF's `LlamaDecoderLayer` and easier to replace it in the code.

    Args:
        config: LlamaConfig
        args: positional args (for compatibility with `LlamaDecoderLayer`)
        kwargs: keyword args (for compatibility with `LlamaDecoderLayer`)
    """
    def __init__(self, config, *args, **kwargs):
        super().__init__(
            hidden_size=config.hidden_size,
            ffn_hidden_size=config.intermediate_size,
            num_attention_heads=config.num_attention_heads,
            bias=False,
            layernorm_epsilon=config.rms_norm_eps,
            hidden_dropout=0,
            attention_dropout=0,
            fuse_qkv_params=False,
            normalization="RMSNorm",
            activation="swiglu",
            attn_input_format="bshd",
        )
        te_rope = RotaryPositionEmbedding(config.hidden_size//config.num_attention_heads)
        self.te_rope_emb = te_rope(max_seq_len=config.max_position_embeddings).cuda()

    def forward(self,
                hidden_states,
                *args,
                attention_mask,
                **kwargs):
        """
        Custom forward to make sure we only pass relevant arguments to the
        forward pass of the `TransformerLayer`. Also, make sure the output
        format matches the output of the HF's `LlamaDecoderLayer`.
        """
        return (super().forward(hidden_states, attention_mask=attention_mask, rotary_pos_emb=self.te_rope_emb),)
```

2. Before creating a `LlamaForCausalLM`, `replace_decoder` context manager is used to monkey-patch `LlamaDecoderLayer` with `TELlamaDecoderLayer`.

```
@contextmanager
def replace_decoder(te_decodder_cls):
    """
    Replace `LlamaDecoderLayer` with custom `TELlamaDecoderLayer`.
    """
    original_llama_decoder_cls = transformers.models.llama.modeling_llama.LlamaDecoderLayer
    transformers.models.llama.modeling_llama.LlamaDecoderLayer = te_decodder_cls
    try:
        yield
    finally:
        transformers.models.llama.modeling_llama.LlamaDecoderLayer = original_llama_decoder_cls
.
.
.
class TELlamaForCausalLM:
    """
    Causal LM created with `LlamaModel`. The underlying `LlamaDecoderLayer`
    class is monkey-patched with `TELlamaDecoderLayer` class before
    initializing the causal LM with `LlamaForCausalLM`.

    Args:
        config: LlamaConfig
    """

    def __new__(cls, config: LlamaConfig):
        with replace_decoder(te_decodder_cls=TELlamaDecoderLayer):
            llama_for_causal_lm = LlamaForCausalLM(config)
        return llama_for_causal_lm
.
.
.
```

3. A custom `pretrained_from_local` method is added that copies the weights from the checkpoint (which is meant for HF Llama implementation) to the modified `TELlamaForCausalLM` by carefully mapping the weights from the `LlamaDecoderLayer` (HF) to `TransformerLayer` (TE). The method `replace_params` maps and copies apt weights from `LlamaDecoderLayer` to the `TransformerLayer`. Refer to the following diagram for more details.

```
def replace_params(hf_state_dict, te_state_dict):
    # collect all layer prefixes to update
    all_layer_prefixes = set()
    for param_key in hf_state_dict.keys():
        layer_prefix_pat = 'model.layers.\d+.'
        m = re.match(layer_prefix_pat, param_key)
        if m is not None:
            all_layer_prefixes.add(m.group())

    for layer_prefix in all_layer_prefixes:
        # When loading weights into models with less number of layers, skip the
        # copy if the corresponding layer doesn't exist in TE model
        if layer_prefix + 'self_attention.layernorm_qkv.layer_norm_weight' in te_state_dict:
            te_state_dict[layer_prefix + 'self_attention.layernorm_qkv.layer_norm_weight'].data[:] = hf_state_dict[layer_prefix + 'input_layernorm.weight'].data[:]

        if layer_prefix + 'self_attention.layernorm_qkv.query_weight' in te_state_dict:
            te_state_dict[layer_prefix + 'self_attention.layernorm_qkv.query_weight'].data[:] = hf_state_dict[layer_prefix + 'self_attn.q_proj.weight'].data[:]

        if layer_prefix + 'self_attention.layernorm_qkv.key_weight' in te_state_dict:
            te_state_dict[layer_prefix + 'self_attention.layernorm_qkv.key_weight'].data[:] = hf_state_dict[layer_prefix + 'self_attn.k_proj.weight'].data[:]
    .
    .
    .

    return all_layer_prefixes
```

<figure align="center">
<img src="media/weight_swap.png" width="70%">
    <figcaption> Fig 7: Replace `LlamaDecoderLayer` with `TransformerLayer`. </figcaption>
</figure>


After initializing the modified Llama model this way, the core decoder layers get changed to `TELlamaDecoderLayer` (wrapper around `TransformerLayer`) as shown in the following output:
```
ModuleList(
  (0-31): 32 x TELlamaDecoderLayer(
    (self_attention): MultiheadAttention(
      (layernorm_qkv): LayerNormLinear()
      (core_attention): DotProductAttention(
        (flash_attention): FlashAttention()
        (fused_attention): FusedAttention()
        (unfused_attention): UnfusedDotProductAttention(
          (scale_mask_softmax): FusedScaleMaskSoftmax()
          (attention_dropout): Dropout(p=0, inplace=False)
        )
      )
      (proj): Linear()
    )
    (layernorm_mlp): LayerNormMLP()
  )
)
```
In summary, the model gets changed as follows with a bigger chunk of the implementation (core decoder layers) coming from TransformerEngine.

<figure align="center">
<img src="media/model_change.png" width="80%">
    <figcaption> Fig 8: Language model after the HF's `LlamaDecoderLayer`s are replaced with TE's `TransformerLayer`s. </figcaption>
</figure>


<div class="alert alert-info">
<b>Note</b>

Let's first run our "TELlama" implementation in `BF16` precision.
</div>

In [4]:
# Minimize the bloat by wrapping all the imports in a function and return
from utils import *


## Default hyperparams, also defined in `utils.py` in class `Hyperparameters`
# hyperparams.dataset_name = "timdettmers/openassistant-guanaco"
# hyperparams.dataset_text_field = "text"
# hyperparams.learning_rate = 1.41e-5
# hyperparams.batch_size = 8
# hyperparams.max_seq_length = 256
# hyperparams.gradient_accumulation_steps = 1
# hyperparams.num_training_steps=10


## !!! `model_name` attr must point to the location of the model weights !!!
hyperparams.model_name = "/ckpt/llama-7bf-hf/" # <== Add model weight location here
hyperparams.mixed_precision = "bf16"


## Init the model and accelerator wrapper
model = init_te_llama_model(hyperparams)
accelerator, model, optimizer, train_dataloader, lr_scheduler = wrap_with_accelerator(model, hyperparams)


## Finetune the model
finetune_model(model, hyperparams, accelerator, train_dataloader, optimizer, lr_scheduler)

[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser

In [7]:
# Train the model
model.train()
completed_steps = 0
total_loss = 0
optimizer.zero_grad()

for _ in range(10):
    if completed_steps >= num_training_steps:
        break
    for step, batch in enumerate(train_dataloader):
        start_time = time.time()
        with accelerator.accumulate(model):
            outputs = model(**batch)
            loss = outputs.loss
            print(f"Step {step}: loss: {loss.item()}, batch shape: {batch['input_ids'].shape}, peak gpu mem: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")
            total_loss += loss.detach().float()
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        if accelerator.sync_gradients:
            completed_steps += 1

        end_time = time.time()
        total_time = end_time - start_time
        print(f"Step {step} time {total_time}")
        accelerator.log({"batch_time": total_time, "input_ids": batch["input_ids"].cpu().numpy(), "attention_mask": batch["attention_mask"].cpu().numpy()})
        start_time = end_time

        if completed_steps >= num_training_steps:
            break

accelerator.end_training()

Step 0: loss: 2.285313367843628, batch shape: torch.Size([8, 256]), peak gpu mem: 24.39 GB
Step 0 time 1.4395570755004883
Step 1: loss: 3.433112144470215, batch shape: torch.Size([8, 256]), peak gpu mem: 63.13 GB
Step 1 time 0.2440946102142334
Step 2: loss: 4.062510967254639, batch shape: torch.Size([8, 256]), peak gpu mem: 63.13 GB
Step 2 time 0.24254989624023438
Step 3: loss: 3.3462915420532227, batch shape: torch.Size([8, 256]), peak gpu mem: 63.13 GB
Step 3 time 0.24259161949157715
Step 4: loss: 3.0718109607696533, batch shape: torch.Size([8, 256]), peak gpu mem: 63.13 GB
Step 4 time 0.24408507347106934
Step 5: loss: 3.7262790203094482, batch shape: torch.Size([8, 256]), peak gpu mem: 63.13 GB
Step 5 time 0.24119019508361816
Step 6: loss: 5.115753650665283, batch shape: torch.Size([8, 256]), peak gpu mem: 63.13 GB
Step 6 time 0.24044084548950195
Step 7: loss: 3.9757368564605713, batch shape: torch.Size([8, 256]), peak gpu mem: 63.13 GB
Step 7 time 0.24254655838012695
Step 8: loss: 

| Models                                                      | Precision | Step Time (or ms per batch) | Speedup (over baseline) |
|-------------------------------------------------------------|-----------|-----------------------------|-------------------------|
| HF (baseline)                                               | BF16      | 288                         | 1                       |
| HF (replace `nn.Linear` with `TE.Linear`)                   | FP8       | 307                         | 0.94                    |
| TE (replace `LlamaDecoderLayer` with `TE.TransformerLayer`) | BF16      | 243                         | 1.19                   

## [Improvement 3] Replace HF's `LlamaDecoderLayer` with TE's `TransformerLayer` (Precision: `FP8`)

Now that most of the HF Llama model implementation (`LlamaDecoderLayer`s) has been swapped with TransformerEngine implementation (`TELlamaDecoderLayer` or `TransformerLayer`), let's see how `FP8` training helps improve performance.

In [4]:
# Minimize the bloat by wrapping all the imports in a function and return
from utils import *


## Default hyperparams, also defined in `utils.py` in class `Hyperparameters`
# hyperparams.dataset_name = "timdettmers/openassistant-guanaco"
# hyperparams.dataset_text_field = "text"
# hyperparams.learning_rate = 1.41e-5
# hyperparams.batch_size = 8
# hyperparams.max_seq_length = 256
# hyperparams.gradient_accumulation_steps = 1
# hyperparams.num_training_steps=10


## !!! `model_name` attr must point to the location of the model weights !!!
hyperparams.model_name = "/ckpt/llama-7bf-hf/" # <== Add model weight location here
hyperparams.mixed_precision = "fp8"


## Init the model and accelerator wrapper
model = init_te_llama_model(hyperparams)
accelerator, model, optimizer, train_dataloader, lr_scheduler = wrap_with_accelerator(model, hyperparams)


## Finetune the model
finetune_model(model, hyperparams, accelerator, train_dataloader, optimizer, lr_scheduler)

[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser_enabled is deprecated and a no-op (function operator())
[W init.cpp:767] Warning: nvfuser is no longer supported in torch script, use _jit_set_nvfuser

In [7]:
# Train the model
model.train()
completed_steps = 0
total_loss = 0
optimizer.zero_grad()

for _ in range(10):
    if completed_steps >= num_training_steps:
        break
    for step, batch in enumerate(train_dataloader):
        start_time = time.time()
        with accelerator.accumulate(model):
            outputs = model(**batch)
            loss = outputs.loss
            print(f"Step {step}: loss: {loss.item()}, batch shape: {batch['input_ids'].shape}, peak gpu mem: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")
            total_loss += loss.detach().float()
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        if accelerator.sync_gradients:
            completed_steps += 1

        end_time = time.time()
        total_time = end_time - start_time
        print(f"Step {step} time {total_time}")
        accelerator.log({"batch_time": total_time, "input_ids": batch["input_ids"].cpu().numpy(), "attention_mask": batch["attention_mask"].cpu().numpy()})
        start_time = end_time

        if completed_steps >= num_training_steps:
            break

accelerator.end_training()

Step 0: loss: 2.2819559574127197, batch shape: torch.Size([8, 256]), peak gpu mem: 26.61 GB
Step 0 time 3.093244791030884
Step 1: loss: 3.3127150535583496, batch shape: torch.Size([8, 256]), peak gpu mem: 64.54 GB
Step 1 time 0.7673726081848145
Step 2: loss: 3.993682384490967, batch shape: torch.Size([8, 256]), peak gpu mem: 64.54 GB
Step 2 time 0.23313021659851074
Step 3: loss: 3.22248911857605, batch shape: torch.Size([8, 256]), peak gpu mem: 64.54 GB
Step 3 time 0.23214364051818848
Step 4: loss: 3.0897724628448486, batch shape: torch.Size([8, 256]), peak gpu mem: 64.54 GB
Step 4 time 0.23146915435791016
Step 5: loss: 3.6932754516601562, batch shape: torch.Size([8, 256]), peak gpu mem: 64.54 GB
Step 5 time 0.23121118545532227
Step 6: loss: 5.0839033126831055, batch shape: torch.Size([8, 256]), peak gpu mem: 64.54 GB
Step 6 time 0.23106837272644043
Step 7: loss: 3.907317638397217, batch shape: torch.Size([8, 256]), peak gpu mem: 64.54 GB
Step 7 time 0.23175501823425293
Step 8: loss: 3

| Models                                                      | Precision | Step Time (or ms per batch) | Speedup (over baseline) |
|-------------------------------------------------------------|-----------|-----------------------------|-------------------------|
| HF (baseline)                                               | BF16      | 288                         | 1                       |
| HF (replace `nn.Linear` with `TE.Linear`)                   | FP8       | 307                         | 0.94                    |
| TE (replace `LlamaDecoderLayer` with `TE.TransformerLayer`) | BF16      | 243                         | 1.19                    |
| TE (replace `LlamaDecoderLayer` with `TE.TransformerLayer`) | FP8       | 231                         | 1.24                    |


## Benchmarks (revisited)
Let's take a look at the summary of the performance numbers with various configurations.

| Models                                                      | Precision | Step Time (or ms per batch) | Speedup (over baseline) |
|-------------------------------------------------------------|-----------|-----------------------------|-------------------------|
| HF (baseline)                                               | BF16      | 288                         | 1                       |
| HF (replace `nn.Linear` with `TE.Linear`)                   | FP8       | 307                         | 0.94                    |
| TE (replace `LlamaDecoderLayer` with `TE.TransformerLayer`) | BF16      | 243                         | 1.19                    |
| TE (replace `LlamaDecoderLayer` with `TE.TransformerLayer`) | FP8       | 231                         | 1.24                    |

When we use larger chunks of layers from TransformerEngine, we see larger performance improvement. Further, when `FP8` precision is enabled, we see even larger speedup as is expected.
